In [1]:
import angr, claripy

In [2]:
binaryName =  '../bin/CrackMe1.exe'
base_addr = 0x400000
startAddress = 0x401102 # start after input
serialLen = 10 # this should be double the length of the name entered

proj = angr.Project(binaryName, main_opts={'base_addr': base_addr})

In [3]:
state = proj.factory.blank_state(addr=startAddress)
state.options.add(angr.options.ZERO_FILL_UNCONSTRAINED_MEMORY)
state.options.add(angr.options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

encodedName = b'0E0303080E' # this is the encoded form of the entered name "marke"

state.memory.store(state.regs.esp, encodedName) # we can store the encoded name anywhere as long as we put the address in eax
state.regs.eax = state.regs.esp # store the address of the encoded name

serial = claripy.BVS('serial', 8 * (serialLen))
serialSymbolLocation = state.regs.esp - 0x20  # put the symbolic serial number somewhere
state.memory.store(serialSymbolLocation, serial)
state.regs.esi = serialSymbolLocation # the address of the symbolic serial number must be in esi

In [4]:
simgr = proj.factory.simgr(state)

crackedBlock = 0x401139
failedBlock = 0x40114E

simgr.explore(find=crackedBlock, avoid=failedBlock) # we want the serial number to be the same string as the encoded name above

<SimulationManager with 1 found, 10 avoid>

In [5]:
solution_state = simgr.found[0]
serialSolution = solution_state.solver.eval(serial)
hex(serialSolution)

'0x30453033303330383045'